# Fire Propagation Segmentation



In [ ]:
pip install git+https://github.com/qubvel/segmentation_models

  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-ggq2vay9
  Running command git clone -q https://github.com/qubvel/segmentation_models /tmp/pip-req-build-ggq2vay9
  Running command git submodule update --init --recursive -q
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-cp36-none-any.whl size=33792 sha256=e69359dbe330c4596ebece3301ef92bc545acc3b13e67a8e0fb9cb8c0657e945
  Stored in directory: /tmp/pip-ephem-wheel-cache-fobwn73z/wheels/49/cf/46/cbb4bb64518c402aea99df9d466f1081450597e653256bbcf4
Successfully built segmentation-models


In [ ]:
pip install tensorflow==2.1

In [ ]:
import os




In [ ]:
import segmentation_models as sm 

Segmentation Models: using `tf.keras` framework.


In [ ]:

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
BACKBONE = 'resnet34'

BATCH_SIZE = 24
CLASSES = ['covid']
LR = 0.0000001
EPOCHS = 20
preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!unzip /content/drive/'My Drive'/image512.zip

In [ ]:
x_train_dir = "/content/image512/images/"
y_train_dir = "/content/image512/masks/"
x_valid_dir ="/content/image512/val/"
y_valid_dir = "/content/image512/valmasks/"


In [ ]:
data_generator2 = ImageDataGenerator(preprocessing_function=preprocess_input)
data_generator = ImageDataGenerator()

In [ ]:
# Train generators

In [ ]:
x_generator = data_generator2.flow_from_directory(directory=x_train_dir,target_size=(512,512),batch_size=24,seed=42,class_mode=None,classes=None)
y_generator = data_generator.flow_from_directory(directory=y_train_dir,target_size=(512,512),batch_size=24,seed=42,class_mode=None,classes=None)

Found 1095 images belonging to 1 classes.
Found 1095 images belonging to 1 classes.


In [ ]:
valx_generator = data_generator2.flow_from_directory(directory=x_valid_dir,target_size=(512,512),batch_size=24,seed=42,class_mode=None,classes=None)
valy_generator = data_generator.flow_from_directory(directory=y_valid_dir,target_size=(512,512),batch_size=24,seed=42,class_mode=None,classes=None)

Found 141 images belonging to 1 classes.
Found 141 images belonging to 1 classes.


In [ ]:
def combine_generator(gen1, gen2):
        while True:
            yield(next(gen1), next(gen2))

#Create new generators
generator = combine_generator(x_generator,y_generator)
valgenerator = combine_generator(valx_generator,valy_generator)

#Create model
model = sm.Unet()
model = sm.Unet('resnet34', classes=1, activation='sigmoid')
optim = keras.optimizers.Adam(LR)

#Metrics
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]


In [ ]:
# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)



In [1]:
#Fit

model.fit(generator,steps_per_epoch=3000,epochs=100,validation_data=valgenerator,validation_steps=15)